In [1]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor

def start_stable_driver():
    options = Options()
    options.page_load_strategy = 'eager'
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("detach", True)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    prefs = {"profile.managed_default_content_settings.images": 2} # Resimleri engelle
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# Global Ayarlar
BASE_URL = "https://www.sikayetvar.com/arcelik/camasir-makinesi"
washing_links = [] # Link havuzu

In [2]:
target_pages = 200 # Yaklaşık 3500 link toplar
current_page = 1 
refresh_limit = 20 

print(f"🚀 Çamaşır Makinesi linkleri toplanıyor. Hedef: {target_pages} sayfa.")

while current_page <= target_pages:
    driver = start_stable_driver()
    next_stop = min(current_page + refresh_limit - 1, target_pages)
    print(f"🌐 Oturum Başladı: Sayfa {current_page} - {next_stop}")
    
    try:
        for p in range(current_page, next_stop + 1):
            success = False
            attempts = 0
            while not success and attempts < 2:
                try:
                    driver.get(f"{BASE_URL}?page={p}")
                    time.sleep(2.5)
                    
                    links = driver.find_elements(By.CSS_SELECTOR, "a.complaint-layer")
                    if len(links) > 0:
                        for l in links:
                            href = l.get_attribute("href")
                            if href and "/arcelik/" in href:
                                if not any(d['url'] == href for d in washing_links):
                                    washing_links.append({"baslik": l.get_attribute("title"), "url": href})
                        
                        if p % 5 == 0:
                            print(f"✅ Sayfa {p} bitti. Havuz: {len(washing_links)} link.")
                        success = True
                    else:
                        print(f"🛑 Sayfa {p} boş! Lütfen Captcha varsa doğrula.")
                        time.sleep(5)
                        attempts += 1
                except:
                    attempts += 1
                    time.sleep(2)
            current_page = p + 1
    except Exception as e:
        print(f"⚠️ Kritik Hata: {e}")
    
    driver.quit()
    print("🧹 Bellek temizlendi, yeni oturuma geçiliyor...")

print(f"🏁 LİNK TOPLAMA BİTTİ! Toplam: {len(washing_links)} link.")

🚀 Çamaşır Makinesi linkleri toplanıyor. Hedef: 200 sayfa.
🌐 Oturum Başladı: Sayfa 1 - 20
✅ Sayfa 5 bitti. Havuz: 120 link.
✅ Sayfa 10 bitti. Havuz: 238 link.
✅ Sayfa 15 bitti. Havuz: 355 link.
✅ Sayfa 20 bitti. Havuz: 472 link.
🧹 Bellek temizlendi, yeni oturuma geçiliyor...
🌐 Oturum Başladı: Sayfa 21 - 40
✅ Sayfa 25 bitti. Havuz: 587 link.
✅ Sayfa 30 bitti. Havuz: 701 link.
✅ Sayfa 35 bitti. Havuz: 815 link.
✅ Sayfa 40 bitti. Havuz: 935 link.
🧹 Bellek temizlendi, yeni oturuma geçiliyor...
🌐 Oturum Başladı: Sayfa 41 - 60
✅ Sayfa 45 bitti. Havuz: 1048 link.
✅ Sayfa 50 bitti. Havuz: 1166 link.
✅ Sayfa 55 bitti. Havuz: 1281 link.
✅ Sayfa 60 bitti. Havuz: 1396 link.
🧹 Bellek temizlendi, yeni oturuma geçiliyor...
🌐 Oturum Başladı: Sayfa 61 - 80
✅ Sayfa 65 bitti. Havuz: 1512 link.
✅ Sayfa 70 bitti. Havuz: 1627 link.
✅ Sayfa 75 bitti. Havuz: 1741 link.
✅ Sayfa 80 bitti. Havuz: 1844 link.
🧹 Bellek temizlendi, yeni oturuma geçiliyor...
🌐 Oturum Başladı: Sayfa 81 - 100
✅ Sayfa 85 bitti. Havuz: 19

In [3]:
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=60, pool_maxsize=60)
session.mount('https://', adapter)

def fetch_washing_detail(item):
    url = item['url']
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    
    for attempt in range(4): # 4 kez deneme
        try:
            time.sleep(random.uniform(0.1, 0.3) * (attempt + 1))
            res = session.get(url, headers=headers, timeout=12)
            
            if res.status_code == 200:
                soup = BeautifulSoup(res.text, 'html.parser')
                start_node = soup.find("div", {"data-ga-element": "Complaint_Content_Start"})
                content_parts = []
                
                if start_node:
                    for sibling in start_node.find_next_siblings():
                        if sibling.name == "div" and sibling.get("data-ga-element") == "Complaint_Content_End":
                            break
                        if sibling.name == "p":
                            content_parts.append(sibling.get_text(strip=True))
                
                full_text = " ".join(content_parts)
                if not full_text:
                    desc_div = soup.find("div", class_="complaint-detail-description")
                    full_text = desc_div.get_text(strip=True) if desc_div else ""

                if len(full_text) > 5:
                    return {"baslik": item["baslik"], "url": url, "aciklama": full_text}
            elif res.status_code == 429:
                time.sleep(8)
        except:
            continue
    return {"baslik": item["baslik"], "url": url, "aciklama": "HATA: Alınamadı."}

# Çalıştır
print(f"📥 {len(washing_links)} adet çamaşır makinesi şikayeti işleniyor...")
final_washing_data = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_washing_detail, link) for link in washing_links]
    for i, future in enumerate(futures, 1):
        final_washing_data.append(future.result())
        
        if i % 100 == 0:
            elapsed = time.time() - start_time
            rem = (elapsed / i) * (len(washing_links) - i) / 60
            print(f"📊 Durum: {i}/{len(washing_links)} | Kalan: ~{rem:.1f} dk")
            
        if i % 200 == 0: # Her 200'de bir yedek al
            pd.DataFrame(final_washing_data).to_excel("arcelik_camasir_YEDEK.xlsx", index=False)



📥 3140 adet çamaşır makinesi şikayeti işleniyor...
📊 Durum: 100/3140 | Kalan: ~21.1 dk
📊 Durum: 200/3140 | Kalan: ~26.8 dk
📊 Durum: 300/3140 | Kalan: ~27.3 dk
📊 Durum: 400/3140 | Kalan: ~26.7 dk
📊 Durum: 500/3140 | Kalan: ~26.5 dk
📊 Durum: 600/3140 | Kalan: ~26.5 dk
📊 Durum: 700/3140 | Kalan: ~24.2 dk
📊 Durum: 800/3140 | Kalan: ~24.3 dk
📊 Durum: 900/3140 | Kalan: ~22.8 dk
📊 Durum: 1000/3140 | Kalan: ~21.7 dk
📊 Durum: 1100/3140 | Kalan: ~20.4 dk
📊 Durum: 1200/3140 | Kalan: ~19.6 dk
📊 Durum: 1300/3140 | Kalan: ~18.9 dk
📊 Durum: 1400/3140 | Kalan: ~17.6 dk
📊 Durum: 1500/3140 | Kalan: ~16.6 dk
📊 Durum: 1600/3140 | Kalan: ~15.4 dk
📊 Durum: 1700/3140 | Kalan: ~14.6 dk
📊 Durum: 1800/3140 | Kalan: ~13.5 dk
📊 Durum: 1900/3140 | Kalan: ~12.4 dk
📊 Durum: 2000/3140 | Kalan: ~11.5 dk
📊 Durum: 2100/3140 | Kalan: ~10.5 dk
📊 Durum: 2200/3140 | Kalan: ~9.6 dk
📊 Durum: 2300/3140 | Kalan: ~8.5 dk
📊 Durum: 2400/3140 | Kalan: ~7.6 dk
📊 Durum: 2500/3140 | Kalan: ~6.6 dk
📊 Durum: 2600/3140 | Kalan: ~5.6 dk
📊

In [4]:
# Final Kayıt
pd.DataFrame(final_washing_data).to_excel("arcelik_camasir_makinesi_YENI.xlsx", index=False)
print("✅ İŞLEM TAMAMLANDI! Yeni dosya: arcelik_camasir_makinesi_YENI.xlsx")

✅ İŞLEM TAMAMLANDI! Yeni dosya: arcelik_camasir_makinesi_YENI.xlsx
